In [2]:
import os
import sys
from watson_developer_cloud import DiscoveryV1
from dotenv import load_dotenv, find_dotenv
import json
import csv
import requests

In [3]:
try:
    load_dotenv(find_dotenv())
except IOError:
    print('warning: no .env file loaded')

In [4]:
discovery = DiscoveryV1(
              #url=os.getenv('DISCOVERY_URL'),
              username=os.getenv('DISCOVERY_USERNAME'),
              password=os.getenv('DISCOVERY_PASSWORD'),
              version="2018-03-05"
            )
environment_id = os.getenv('DISCOVERY_ENVIRONMENT_ID')
configuration_id = os.getenv('DISCOVERY_CONFIGURATION_ID')
collection_id =  os.getenv('DISCOVERY_COLLECTION_ID')
# collection_id = '6cd34257-ac8c-43d5-b62f-61e0e9732eca'
print(' environment_id :',environment_id,'\n configuration_id : ',configuration_id,'\n collection_id : ',collection_id)                                 

 environment_id : 162e8398-ceff-42ab-8710-655087535953 
 configuration_id :  0806bbdd-6bef-4882-8ba9-093c37749337 
 collection_id :  6cd34257-ac8c-43d5-b62f-61e0e9732eca


In [17]:
params = {}
params["query"] = "%s" % ('How to carry a patient?')

#discovery.query(environment_id=environment_id, collection_id=collection_id, query_options=params, passage=True)
discovery.query(environment_id=environment_id, collection_id=collection_id, 
                natural_language_query='How to carry a patient?', passage=True,
                passages_count=5
               )



{'matching_results': 11,
 'passages': [{'document_id': '9f1d703ad82c403b100bca8b2514353b',
   'end_offset': 458,
   'field': 'html',
   'passage_score': 15.344720048868776,
   'passage_text': '>\n\nHow to carry a patient?\nhttps://www.youtube.com/watch?v=F2FSkBJyDLw\n\nFirst-Aid Tips - Transport of the Injured \nhttps://www.youtube.com/watch?v=ApzANyz15KI\n\nHow to Lift &amp; Move an Accident Victim \nhttps://www.youtube.com/watch?v=94XBLKQtJYw\n\n</body></html>',
   'start_offset': 196},
  {'document_id': '9f1d703ad82c403b100bca8b2514353b',
   'end_offset': 261,
   'field': 'text',
   'passage_score': 15.109955065778308,
   'passage_text': 'Transport injured person\n\nHow to carry a patient? https://www.youtube.com/watch?v=F2FSkBJyDLw First-Aid Tips - Transport of the Injured https://www.youtube.com/watch?v=ApzANyz15KI How to Lift & Move an Accident Victim https://www.youtube.com/watch?v=94XBLKQtJYw',
   'start_offset': 0},
  {'document_id': '7387690f25c73eb6ca89716bd353beff',
   'end

# Collect questions and get the response from discovery without relevancy training

In [19]:
with open ("/Users/rajeshgudikoti/Documents/github_prj/cfc-first-aid/questionnaire.txt") as questions:
    #open an output file to place the responses 
    filestr = "/Users/rajeshgudikoti/Documents/github_prj/cfc-first-aid/response.csv"
    of = open(filestr, "w")
    writer = csv.writer(of, delimiter=",")
    
    #go through each question in file and prepare Discovery query paramaters 
    for line in questions:
        question = line.replace("\n", "")
        params = {}
        params["query"] = "%s" % (question)
        print('\n\n qqqqq qqqqqqq qqqqqqqqq qqqqqqqqq qqqqqqqq')
        print(question,' \t\t ',params["query"])
        #params["return"] = {"_id","body","title"} #these fields may need to be updated depending on the content being used 
        #params["return"] = "_id" #these fields may need to be updated depending on the content being used 
        key = "return"
        params.setdefault(key, [])
        #a[key].append(1)
        params["return"].append("_id")
        params["return"].append("body")
        params["return"].append("title")
        params["count"] = 4 
        #print('params["return"] ',params["return"])
        #run Discovery query to get results from untrained service 
        
        #result = discovery.query(environment_id=environment_id, collection_id=collection_id, query_options=params)
        result = discovery.query(environment_id=environment_id, collection_id=collection_id, 
                                 natural_language_query=question,passage=True,
                                 passages_count=5)
        
        #create a row for each query and results 
        result_list = [question.encode("utf8")]
        print('length of resultset',len(result["results"]))
        for resultDoc in result["results"]:
            #print('resultDoc_id',resultDoc['id'])
            
            #print("resultDoc.get('extracted_metadata').get('title') " ,resultDoc.get('extracted_metadata').get('title'))
            id = resultDoc["id"]
            print('\n resultDoc.extracted_metadata \n', resultDoc.get('extracted_metadata'))
            #if('extracted_metadata'["title"] in resultDoc):
            #if(resultDoc.get('extracted_metadata').get('title') is not 'None' or 'no title'):
            if('title' in resultDoc.get('extracted_metadata')):
                print('title --> ', resultDoc['extracted_metadata']['title'])
                title = resultDoc['extracted_metadata']['title'].encode("utf8")
            else:
                title = 'title not present'
            #if(resultDoc.get('html') is not 'None'):
            if('html' in resultDoc):
                body = resultDoc['html'].encode("utf8")
                #print('html present in resultDoc \n-->')
            if('text' in resultDoc):
                body = resultDoc['text'].encode("utf8")
                #print('text present in resultDoc \n-->')
            else:
                body ='body not present'
                #print('body not present in resultDoc \n-->' , resultDoc)
            
            print('\n\n *********************************************')
            print('id \n', id, '\n title \n',title)
            
            #result_list.extend([id,title,body,' ']) #leave a space to enter a relevance label for each doc 
            result_list.extend([id,title,' '])
            
        #write the row to the file 
        writer.writerow(result_list)
    
    of.close()
    #print('result_list',result_list)



 qqqqq qqqqqqq qqqqqqqqq qqqqqqqqq qqqqqqqq
what are dos and donts?  		  what are dos and donts?
length of resultset 10

 resultDoc.extracted_metadata 
 {'publicationdate': '2010-06-23', 'sha1': '0900dfbfd8e59eed96f94f2568c27b7a2e204573', 'filename': 'FS_disaster_toolkit.pdf', 'file_type': 'pdf', 'title': 'no title'}
title -->  no title


 *********************************************
id 
 c148148b6ecf9b218d1b58e3b4d1a572 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2010-06-23', 'sha1': '0900dfbfd8e59eed96f94f2568c27b7a2e204573', 'filename': 'DisERHandbook.pdf', 'file_type': 'pdf', 'title': 'no title'}
title -->  no title


 *********************************************
id 
 4dac8de2-a6ed-4929-8a23-7abb72711e0e 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2009-01-29', 'sha1': 'f465d1cf2d602d86af5a0c64d74b198aaf8b54e2', 'author': 'Flora Or', 'filename': 'disasterpreventionpreparedness.pdf', 'file_type': 'pdf', 'title': 'Mic

length of resultset 10

 resultDoc.extracted_metadata 
 {'sha1': '34a0af3b9656e2a1839069b023756c85ab41a431', 'filename': 'filename', 'file_type': 'json'}


 *********************************************
id 
 493fed166c4344bbc5512ac068975514e3cda6b9efb908b2910b204dec4b0d06 
 title 
 title not present

 resultDoc.extracted_metadata 
 {'publicationdate': '2010-06-23', 'sha1': '0900dfbfd8e59eed96f94f2568c27b7a2e204573', 'filename': 'FS_disaster_toolkit.pdf', 'file_type': 'pdf', 'title': 'no title'}
title -->  no title


 *********************************************
id 
 c148148b6ecf9b218d1b58e3b4d1a572 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2010-06-23', 'sha1': '0900dfbfd8e59eed96f94f2568c27b7a2e204573', 'filename': 'DisERHandbook.pdf', 'file_type': 'pdf', 'title': 'no title'}
title -->  no title


 *********************************************
id 
 4dac8de2-a6ed-4929-8a23-7abb72711e0e 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicatio

length of resultset 9

 resultDoc.extracted_metadata 
 {'sha1': 'a96971bef12750bca993eff464e724291232af2c', 'filename': 'filename', 'file_type': 'json'}


 *********************************************
id 
 71a1c56155c97f88422fe4dd7d14b6d49a4376d985ebe21e55885cca2e38e0d7 
 title 
 title not present

 resultDoc.extracted_metadata 
 {'publicationdate': '2010-06-23', 'sha1': '0900dfbfd8e59eed96f94f2568c27b7a2e204573', 'filename': 'FS_disaster_toolkit.pdf', 'file_type': 'pdf', 'title': 'no title'}
title -->  no title


 *********************************************
id 
 c148148b6ecf9b218d1b58e3b4d1a572 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2010-06-23', 'sha1': '0900dfbfd8e59eed96f94f2568c27b7a2e204573', 'filename': 'DisERHandbook.pdf', 'file_type': 'pdf', 'title': 'no title'}
title -->  no title


 *********************************************
id 
 4dac8de2-a6ed-4929-8a23-7abb72711e0e 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'sha1': 'd27

length of resultset 10

 resultDoc.extracted_metadata 
 {'publicationdate': '2010-06-23', 'sha1': '0900dfbfd8e59eed96f94f2568c27b7a2e204573', 'filename': 'FS_disaster_toolkit.pdf', 'file_type': 'pdf', 'title': 'no title'}
title -->  no title


 *********************************************
id 
 c148148b6ecf9b218d1b58e3b4d1a572 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2010-06-23', 'sha1': '0900dfbfd8e59eed96f94f2568c27b7a2e204573', 'filename': 'DisERHandbook.pdf', 'file_type': 'pdf', 'title': 'no title'}
title -->  no title


 *********************************************
id 
 4dac8de2-a6ed-4929-8a23-7abb72711e0e 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2006-07-10', 'sha1': 'af7c627a1d73779b24aede675fd1f127ec8fb44b', 'author': 'tissotp', 'filename': 'CD_Disasters_26_06_epidemic_disease.pdf', 'file_type': 'pdf', 'title': 'Microsoft Word - CD_Disasters_26_June.doc'}
title -->  Microsoft Word - CD_Disasters_26_June.doc



length of resultset 9

 resultDoc.extracted_metadata 
 {'publicationdate': '2012-06-22', 'sha1': '6daad6f1619e10200b1c9f679f0e28017161b5b1', 'filename': 'epidemics_after_natural_disasters.pdf', 'file_type': 'pdf', 'title': 'no title'}
title -->  no title


 *********************************************
id 
 81f683e92a0d37d5024e6572fb652ca9 
 title 
 b'no title'

 resultDoc.extracted_metadata 
 {'publicationdate': '2009-01-29', 'sha1': 'f465d1cf2d602d86af5a0c64d74b198aaf8b54e2', 'author': 'Flora Or', 'filename': 'disasterpreventionpreparedness.pdf', 'file_type': 'pdf', 'title': 'Microsoft Word - lecnote_fm_disaster prevention and preparedness'}
title -->  Microsoft Word - lecnote_fm_disaster prevention and preparedness


 *********************************************
id 
 13059038802e4bc4a1ef8b0a8db1309c 
 title 
 b'Microsoft Word - lecnote_fm_disaster prevention and preparedness'

 resultDoc.extracted_metadata 
 {'publicationdate': '2006-07-10', 'sha1': 'af7c627a1d73779b24aede675fd1f12

...

length of resultset 10

 resultDoc.extracted_metadata 
 {'sha1': 'fca12e3bbeac04104612d174e15b59e5ec2424e5', 'filename': 'first_aid_back_injury.html', 'file_type': 'html', 'title': '\n\nFirst aid for back injury\n\n'}
title -->  

First aid for back injury




 *********************************************
id 
 41ac968e4b9cfc2a04d2f1a1782a0a41 
 title 
 b'\n\nFirst aid for back injury\n\n'

 resultDoc.extracted_metadata 
 {'sha1': '3de2e1337344c8d3b5eda66277a4f8f441a1e4a1', 'filename': 'first_aid_neck_injury.html', 'file_type': 'html', 'title': '\n\nFirst aid for neck injury\nhttps://www.youtube.com/watch?v=BylrfShDn3E&feature=youtu.be\n\n'}
title -->  

First aid for neck injury
https://www.youtube.com/watch?v=BylrfShDn3E&feature=youtu.be




 *********************************************
id 
 7387690f25c73eb6ca89716bd353beff 
 title 
 b'\n\nFirst aid for neck injury\nhttps://www.youtube.com/watch?v=BylrfShDn3E&feature=youtu.be\n\n'

 resultDoc.extracted_metadata 
 {'sha1': '3fb1ba5b5

length of resultset 10

 resultDoc.extracted_metadata 
 {'sha1': 'fca12e3bbeac04104612d174e15b59e5ec2424e5', 'filename': 'first_aid_back_injury.html', 'file_type': 'html', 'title': '\n\nFirst aid for back injury\n\n'}
title -->  

First aid for back injury




 *********************************************
id 
 41ac968e4b9cfc2a04d2f1a1782a0a41 
 title 
 b'\n\nFirst aid for back injury\n\n'

 resultDoc.extracted_metadata 
 {'publicationdate': '2009-01-29', 'sha1': 'f465d1cf2d602d86af5a0c64d74b198aaf8b54e2', 'author': 'Flora Or', 'filename': 'disasterpreventionpreparedness.pdf', 'file_type': 'pdf', 'title': 'Microsoft Word - lecnote_fm_disaster prevention and preparedness'}
title -->  Microsoft Word - lecnote_fm_disaster prevention and preparedness


 *********************************************
id 
 13059038802e4bc4a1ef8b0a8db1309c 
 title 
 b'Microsoft Word - lecnote_fm_disaster prevention and preparedness'

 resultDoc.extracted_metadata 
 {'sha1': '3de2e1337344c8d3b5eda66277a4f8f441a

length of resultset 10

 resultDoc.extracted_metadata 
 {'sha1': 'd5bb4ae8bb1a9c2d51106a1962f800a8697e1c7e', 'filename': 'first_aid_spine_injury.html', 'file_type': 'html', 'title': '\nFirst Aid for spine injury\n'}
title -->  
First Aid for spine injury



 *********************************************
id 
 5485adcbda917562d4076f7f86216f92 
 title 
 b'\nFirst Aid for spine injury\n'

 resultDoc.extracted_metadata 
 {'publicationdate': '2009-01-29', 'sha1': 'f465d1cf2d602d86af5a0c64d74b198aaf8b54e2', 'author': 'Flora Or', 'filename': 'disasterpreventionpreparedness.pdf', 'file_type': 'pdf', 'title': 'Microsoft Word - lecnote_fm_disaster prevention and preparedness'}
title -->  Microsoft Word - lecnote_fm_disaster prevention and preparedness


 *********************************************
id 
 13059038802e4bc4a1ef8b0a8db1309c 
 title 
 b'Microsoft Word - lecnote_fm_disaster prevention and preparedness'

 resultDoc.extracted_metadata 
 {'publicationdate': '2010-06-23', 'sha1': '0900dfbf

length of resultset 10

 resultDoc.extracted_metadata 
 {'sha1': 'd5bb4ae8bb1a9c2d51106a1962f800a8697e1c7e', 'filename': 'first_aid_spine_injury.html', 'file_type': 'html', 'title': '\nFirst Aid for spine injury\n'}
title -->  
First Aid for spine injury



 *********************************************
id 
 5485adcbda917562d4076f7f86216f92 
 title 
 b'\nFirst Aid for spine injury\n'

 resultDoc.extracted_metadata 
 {'sha1': '27e9df9b119d055db704dcd0f1fedacfdd4f58f8', 'filename': 'transport_injured_person.html', 'file_type': 'html', 'title': '\nTransport injured person\n'}
title -->  
Transport injured person



 *********************************************
id 
 9f1d703ad82c403b100bca8b2514353b 
 title 
 b'\nTransport injured person\n'

 resultDoc.extracted_metadata 
 {'sha1': '3de2e1337344c8d3b5eda66277a4f8f441a1e4a1', 'filename': 'first_aid_neck_injury.html', 'file_type': 'html', 'title': '\n\nFirst aid for neck injury\nhttps://www.youtube.com/watch?v=BylrfShDn3E&feature=youtu.be\n\

length of resultset 10

 resultDoc.extracted_metadata 
 {'sha1': '3de2e1337344c8d3b5eda66277a4f8f441a1e4a1', 'filename': 'first_aid_neck_injury.html', 'file_type': 'html', 'title': '\n\nFirst aid for neck injury\nhttps://www.youtube.com/watch?v=BylrfShDn3E&feature=youtu.be\n\n'}
title -->  

First aid for neck injury
https://www.youtube.com/watch?v=BylrfShDn3E&feature=youtu.be




 *********************************************
id 
 7387690f25c73eb6ca89716bd353beff 
 title 
 b'\n\nFirst aid for neck injury\nhttps://www.youtube.com/watch?v=BylrfShDn3E&feature=youtu.be\n\n'

 resultDoc.extracted_metadata 
 {'sha1': '27e9df9b119d055db704dcd0f1fedacfdd4f58f8', 'filename': 'transport_injured_person.html', 'file_type': 'html', 'title': '\nTransport injured person\n'}
title -->  
Transport injured person



 *********************************************
id 
 9f1d703ad82c403b100bca8b2514353b 
 title 
 b'\nTransport injured person\n'

 resultDoc.extracted_metadata 
 {'sha1': '3fb1ba5b5a1ca4c5687

# Upload Training Data

In [5]:
#function for posting to training data endpoint 
def training_post(discovery_training_path, training_obj):
    #training_json = json.dumps(training_obj)
    training_json = training_obj
    headers = {
        "content-type": "application/json",
        "Accept" : "text/plain"
        }
    auth = (os.getenv('DISCOVERY_USERNAME'), os.getenv('DISCOVERY_PASSWORD'))
    
    response = requests.request(method="POST",url=discovery_training_path,data=training_json,headers=headers,auth=auth)
    #print(response.status_code , response.reason, '\n', response.request, response.url,response.text)
    print('\n\n',response.reason,response.text,'\n\n')
   

In [6]:
#open the training file and create new training data objects
with open('/Users/rajeshgudikoti/Documents/github_prj/cfc-first-aid/response.csv','r') as training_doc:
    training_csv = csv.reader(training_doc, delimiter=',')    
    training_obj = {}
    training_obj["examples"] = []
    
    discovery_path = "https://gateway.watsonplatform.net/discovery/api/v1/environments/" + environment_id + "/collections/" + collection_id 
    discovery_training_path = discovery_path + "/training_data?version=2016-11-07"
    
    for row in training_csv:
        training_obj["natural_language_query"] = row[0]
        i = 1 
        print('\nrow ', row[0])
        training_obj["examples"]=[]
        #training_obj.fromkeys(["examples"])
        print('************\n traning example before loop*******', training_obj["examples"])
        for j in range(1,10):
            example_obj = {}
            
            #example_obj["relevance"] = row[i+3]
            # becuase we do not include body in result...see above cell
            example_obj["relevance"] = row[i+2]
            example_obj["document_id"] = row[i]
            
            print('example_obj.relevance', example_obj["relevance"])
            if (example_obj["relevance"]) is not ' ' or (example_obj["relevance"]) == 0:
                training_obj["examples"].append(example_obj)
            else:
                print('example_obj.relevance is zero or not specified')
                example_obj["relevance"] = 0
            #if (example_obj["relevance"]) is not ' ':
            #if (example_obj["relevance"]) != "0":
                #training_obj["examples"].append(example_obj)
            #else:
                #print('example_obj.relevance is zero')
            
            training_obj["examples"].append(example_obj)
            
            #print('example_obj', example_obj)
            #training_obj["examples"] = example_obj
            i = i + 3
            
            #print('relevance object ',example_obj["relevance"] )
            #print('document id ', example_obj["document_id"])
            #print('traning example', training_obj["examples"])
            #print('***** training_obj', json.dumps(training_obj))
        
            #send the training data to the discovery service 
        response = training_post(discovery_training_path, json.dumps(training_obj))
            #print(response.status_code , response.reason, '\n', response.request, response.url,response.text)
            
            #print('traning example', training_obj["examples"])


row  b'what are dos and donts?'
************
 traning example before loop******* []
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance 5
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance 5
example_obj.relevance 5
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified


 Created {"natural_language_query":"b'what are dos and donts?'","filter":"","examples":[{"document_id":"4dac8de2-a6ed-4929-8a23-7abb72711e0e","cross_reference":"","relevance":0,"created":"2018-08-30T04:40:12.681Z","updated":"2018-08-30T04:40:12.697Z"},{"document_id":"0063cb29203a8d7bc01fd47c6405d650b9f1d4b9967f8fa6fec80a57431596e4","cross_reference":"","relevance":5,"created":"2018-08-30T04:40:12.681Z","updated"



 Created {"natural_language_query":"b'what are dos and donts during fire?'","filter":"","examples":[{"document_id":"4dac8de2-a6ed-4929-8a23-7abb72711e0e","cross_reference":"","relevance":0,"created":"2018-08-30T04:40:16.429Z","updated":"2018-08-30T04:40:16.462Z"},{"document_id":"0063cb29203a8d7bc01fd47c6405d650b9f1d4b9967f8fa6fec80a57431596e4","cross_reference":"","relevance":0,"created":"2018-08-30T04:40:16.429Z","updated":"2018-08-30T04:40:16.465Z"},{"document_id":"ecad0f21582c9353daa2596feae0c35002f8b380553a333257c06c9dbdf4bc9d","cross_reference":"","relevance":0,"created":"2018-08-30T04:40:16.429Z","updated":"2018-08-30T04:40:16.469Z"},{"document_id":"71a1c56155c97f88422fe4dd7d14b6d49a4376d985ebe21e55885cca2e38e0d7","cross_reference":"","relevance":0,"created":"2018-08-30T04:40:16.429Z","updated":"2018-08-30T04:40:16.472Z"},{"document_id":"493fed166c4344bbc5512ac068975514e3cda6b9efb908b2910b204dec4b0d06","cross_reference":"","relevance":0,"created":"2018-08-30T04:40:16.429Z","upd



 Conflict {"details":null,"error":"ALREADY_EXISTS: Query string 'b'Can I have food?'||' id f76941592585bc01775a239c8f206ec8b5a858bd already exists in collection 6cd34257-ac8c-43d5-b62f-61e0e9732eca, environment 162e8398-ceff-42ab-8710-655087535953","code":409} 



row  b'Can I eat?'
************
 traning example before loop******* []
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance 5


 Conflict {"details":null,"error":"ALREADY_EXISTS: Query string 'b'C



 Conflict {"details":null,"error":"ALREADY_EXISTS: Query string 'b'How to prepare post floods?'||' id 8981bb2dcd0d436f444115f5d304091fba6bb425 already exists in collection 6cd34257-ac8c-43d5-b62f-61e0e9732eca, environment 162e8398-ceff-42ab-8710-655087535953","code":409} 



row  b'What measures should we take post floods?'
************
 traning example before loop******* []
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance 4
example_obj.relevance 4
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance 5
example_obj.relevance 5
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified


 Conflict {"details":null,"error":"ALREADY_EXISTS: Query string 'b'What measures should we take post floods?'||' id 5be67042f6f364880f4945e7e03637f927fb4d01 already ex



 Conflict {"details":null,"error":"ALREADY_EXISTS: Query string 'b'What items we should keep as emergency during natural calamities? '||' id f4db5b4343d6ea7cbf05320c78fadf61230794d0 already exists in collection 6cd34257-ac8c-43d5-b62f-61e0e9732eca, environment 162e8398-ceff-42ab-8710-655087535953","code":409} 



row  b'What items we should keep as emergency during cyclones? '
************
 traning example before loop******* []
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance 5
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance 5
example_obj.relevance 5
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified


 Conflict {"details":null,"error":"ALREADY_EXISTS: Query string 'b



 Conflict {"details":null,"error":"ALREADY_EXISTS: Query string 'b'Do I need any emergency kit fire? '||' id 9a44d5f04ba6a93c8a8dfda0e56b523cc3fe1b2a already exists in collection 6cd34257-ac8c-43d5-b62f-61e0e9732eca, environment 162e8398-ceff-42ab-8710-655087535953","code":409} 



row  b'Can I drink water post floods?'
************
 traning example before loop******* []
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance 5
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance 5
example_obj.relevance 5
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified


 Conflict {"details":null,"error":"ALREADY_EXISTS: Query string 'b'Can I drink water post floods?'||' id f75c67e71e872069e11



 Conflict {"details":null,"error":"ALREADY_EXISTS: Query string 'b'How do we manage Severe bleeding due to injury during natural calamities?'||' id d1d25859e45ccb3562937b421b4b78b1896fabad already exists in collection 6cd34257-ac8c-43d5-b62f-61e0e9732eca, environment 162e8398-ceff-42ab-8710-655087535953","code":409} 



row  b'Help Choking Adult or child over 1 year?'
************
 traning example before loop******* []
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example_obj.relevance  
example_obj.relevance is zero or not specified
example



 Conflict {"details":null,"error":"ALREADY_EXISTS: Query string 'b'First aid for asthma attack?'||' id 4399c27b25faa52f76fbf0d354da9b76854f9af0 already exists in collection 6cd34257-ac8c-43d5-b62f-61e0e9732eca, environment 162e8398-ceff-42ab-8710-655087535953","code":409} 




# Query post relevance training

In [42]:
response= discovery.query(environment_id, collection_id, filter="enriched_text.entities.type::NaturalEvent", query=None, natural_language_query='First aid for asthma attack?', 
                passages=True,
                #aggregation= 'term(classname).top_hits(5)', 
      count=None, return_fields=None, offset=None, sort=None, highlight=None, 
      passages_fields=None, passages_count=None, passages_characters=None, deduplicate=None, deduplicate_field=None)
print(len(response['passages']))
for items in response['passages']:
    #print(response['passages'][0].get('passage_text').find('aid'))
    if(items.get('passage_text').find('aid')>-1 and items.get('passage_text').find('asthma')):
        print(items.get('document_id'))
        print(items.get('passage_score'))
        print(items.get('passage_text'))


10
13059038802e4bc4a1ef8b0a8db1309c
10.762155251732835
</p><p>• Be aware of possible tsunamis if you live in coastal areas. </p><p><b>Public health interventions</b>: </p><p>• Encourage earthquake drills to practice emergency procedures; </p><p>• Recommend items for inclusion in an extensive first aid kit and survival
81f683e92a0d37d5024e6572fb652ca9
10.440950557893967
Cross-cultural dedication to common good is most common response to natural disasters.

3. Thousands volunteer to rescue strangers and sift through rubble after earthquakes from Mexico City, California, and Turkey.

4. Most rescue, first aid, and transport are from other casualties and bystanders.
13059038802e4bc4a1ef8b0a8db1309c
10.440950557893967
Emergency phase: activities should focus on saving lives through search and rescue, first aid, emergency medical assistance and over all disaster assessment. Efforts immediately begin to repair critical facilities, to restore communications, and transportation networks, and in

# Delete All Training Data

In [ ]:
discovery.delete_all_training_data(environment_id, collection_id)

# Add a new document

In [84]:
with open(os.path.join(os.getcwd(), '/Users/rajeshgudikoti/Documents/github_prj/cfc-first-aid/disco-docs/first_aid/transport_injured_person.html'),'rb') as fileinfo:
    add_doc = discovery.add_document(environment_id, collection_id, file=fileinfo)
print(json.dumps(add_doc, indent=2))

{
  "document_id": "277847db-32b8-4d9f-8d29-9207ee4236ea",
  "status": "processing"
}


In [85]:
document_id='277847db-32b8-4d9f-8d29-9207ee4236ea'
print(add_doc['status'])
doc_info = discovery.get_document_status(environment_id, collection_id, document_id)
print(json.dumps(doc_info, indent=2))

processing
{
  "document_id": "277847db-32b8-4d9f-8d29-9207ee4236ea",
  "notices": [],
  "status": "available",
  "status_description": "Document is successfully ingested and indexed with no warnings",
  "filename": "transport_injured_person.html",
  "file_type": "html",
  "sha1": "b0395d882741ea7eb341b3d2ea6d029499da8bfd"
}


# Update a document title

In [86]:
document_id = '277847db-32b8-4d9f-8d29-9207ee4236ea'
with open(os.path.join(os.getcwd(), '/Users/rajeshgudikoti/Documents/github_prj/cfc-first-aid/disco-docs/first_aid/transport_injured_person.html'),'rb') as fileinfo:
    add_updated_doc = discovery.update_document(environment_id, collection_id, document_id,
                                                file=fileinfo, 
                                                file_content_type='application/html',
                                                filename='transport_injured_person.html',
                                                title='Transport injured person'
                                                #,
                                                #metadata = {'title':"epidemics_after_natural_disasters"}
                                                )
print(json.dumps(add_updated_doc, indent=2))

{
  "document_id": "277847db-32b8-4d9f-8d29-9207ee4236ea",
  "status": "processing"
}


In [88]:
print(add_updated_doc['status'])
document_id = add_updated_doc['document_id']
print(document_id)
doc_info = discovery.get_document_status(environment_id, collection_id, document_id)
print(json.dumps(doc_info, indent=2))

processing
277847db-32b8-4d9f-8d29-9207ee4236ea
{
  "document_id": "277847db-32b8-4d9f-8d29-9207ee4236ea",
  "notices": [],
  "status": "available",
  "status_description": "Document is successfully ingested and indexed with no warnings",
  "filename": "transport_injured_person.html",
  "file_type": "html",
  "sha1": "b0395d882741ea7eb341b3d2ea6d029499da8bfd"
}


# Delete a document

In [83]:
delete_doc = discovery.delete_document(environment_id, collection_id, '9f1d703ad82c403b100bca8b2514353b')
print(json.dumps(delete_doc, indent=2))

{
  "document_id": "9f1d703ad82c403b100bca8b2514353b",
  "status": "deleted"
}


# Add training data + examples for individual NLQ

In [8]:
data = {
    "natural_language_query": "what are dos and donts during earthquake", 
        
    "filter": "test2", 
    "examples": [{"document_id": "493fed166c4344bbc5512ac068975514e3cda6b9efb908b2910b204dec4b0d0", "relevance": 2}, 
                 {"document_id": "18ed484753a7f9845b364bdf74f2eb79", "relevance": 1}]
}

headers = {
        "content-type": "application/json",
        "Accept" : "text/plain"
        }

discovery_path = "https://gateway.watsonplatform.net/discovery/api/v1/environments/" + environment_id + "/collections/" + collection_id + "/training_data"
discovery_training_path = discovery_path + "?version=2018-03-05"

username = os.getenv('DISCOVERY_USERNAME')
password = os.getenv('DISCOVERY_PASSWORD')

auth = (username, password)

print(discovery_path, '\n', discovery_training_path)
print(data)
#data = json.dumps(data)
#print(data)

response = requests.request(method="POST",url=discovery_training_path,json=data,headers=headers,auth=auth)
#response = requests.post(url=discovery_path,params=params,json=data,headers=headers)
print(response.json())

#add_training_data(discovery, environment_id, collection_id, natural_language_query='what are dos and donts during earthquake', filter=None, 
                  #examples=examples, **kwargs)
print(response.status_code , response.reason, '\n', response.request, response.url,response.text)

https://gateway.watsonplatform.net/discovery/api/v1/environments/162e8398-ceff-42ab-8710-655087535953/collections/6cd34257-ac8c-43d5-b62f-61e0e9732eca/training_data 
 https://gateway.watsonplatform.net/discovery/api/v1/environments/162e8398-ceff-42ab-8710-655087535953/collections/6cd34257-ac8c-43d5-b62f-61e0e9732eca/training_data?version=2018-03-05
{'natural_language_query': 'what are dos and donts during earthquake', 'filter': 'test2', 'examples': [{'document_id': '493fed166c4344bbc5512ac068975514e3cda6b9efb908b2910b204dec4b0d0', 'relevance': 2}, {'document_id': '18ed484753a7f9845b364bdf74f2eb79', 'relevance': 1}]}
{'natural_language_query': 'what are dos and donts during earthquake', 'filter': 'test2', 'examples': [{'document_id': '493fed166c4344bbc5512ac068975514e3cda6b9efb908b2910b204dec4b0d0', 'cross_reference': '', 'relevance': 2, 'created': '2018-08-08T05:17:16.284Z', 'updated': '2018-08-08T05:17:16.306Z'}, {'document_id': '18ed484753a7f9845b364bdf74f2eb79', 'cross_reference': '

In [13]:
json_output = json.loads(response.text)
query_id=json_output['query_id']
print(query_id)

8bd44b5c90fafd50a7cff98618c2837211c6c79f


# Add example to existing query

In [17]:
query_id = 8bd44b5c90fafd50a7cff98618c2837211c6c79f
data = {
   "document_id": "493fed166c4344bbc5512ac068975514e3cda6b9efb908b2910b204dec4b0d06",
   "relevance": 1
 }
json_output = json.loads(response.text)
add_example_path = discovery_path + "/" + query_id + "/examples?version=2018-03-05"
print(add_example_path)
response = requests.request(method="POST",url=add_example_path,json=data,headers=headers,auth=auth)
print(response.status_code , response.reason, '\n', response.request, response.url,response.text)

https://gateway.watsonplatform.net/discovery/api/v1/environments/162e8398-ceff-42ab-8710-655087535953/collections/6cd34257-ac8c-43d5-b62f-61e0e9732eca/training_data/8bd44b5c90fafd50a7cff98618c2837211c6c79f/examples?version=2018-03-05
201 Created 
 <PreparedRequest [POST]> https://gateway.watsonplatform.net/discovery/api/v1/environments/162e8398-ceff-42ab-8710-655087535953/collections/6cd34257-ac8c-43d5-b62f-61e0e9732eca/training_data/8bd44b5c90fafd50a7cff98618c2837211c6c79f/examples?version=2018-03-05 {"document_id":"493fed166c4344bbc5512ac068975514e3cda6b9efb908b2910b204dec4b0d06","cross_reference":"","relevance":1,"created":"2018-08-08T05:23:54.384Z","updated":"2018-08-08T05:23:54.530Z"}


In [13]:
# Dictionary of string and int
wordFreqDic = {
    "Hello": 56,
    "at" : 23 ,
    "test" : 43,
    "this" : 78,
    "test2" :{
        "nkey1": "value1"
    }
}

In [16]:
if "test2" in wordFreqDic:
    print("Yes 'test2' key exists in dict")
else:
    print("No 'test' key does not exists in dict") 
if ("nkey1") in wordFreqDic:
 print("Yes 'nkey1' key exists in dict")
else:
    print("No 'nkey1' key does not exists in dict")
print(wordFreqDic.get('test2'))
if('nkey1' in wordFreqDic.get('test2')):
    print("Yes 'nkey1' key exists in dict")
else:
    print("No 'nkey1' key does not exists in dict")

Yes 'test2' key exists in dict
No 'nkey1' key does not exists in dict
{'nkey1': 'value1'}
Yes 'nkey1' key exists in dict


In [6]:
import requests
headers = {
        "content-type": "application/json",
        "Accept" : "text/plain"
        }
data = {
    "natural_language_query": "what are dos and donts during earthquake", 
        
    "filter": "test2", 
    "examples": [{"document_id": "493fed166c4344bbc5512ac068975514e3cda6b9efb908b2910b204dec4b0d0", "relevance": 2}, 
                 {"document_id": "18ed484753a7f9845b364bdf74f2eb79", "relevance": 1}]
}
req = requests.Request('POST','https://gateway.watsonplatform.net/discovery/api/v1/environments/162e8398-ceff-42ab-8710-655087535953/collections/6cd34257-ac8c-43d5-b62f-61e0e9732eca/training_data?version=2018-03-05',headers=headers,data=data)
prepared = req.prepare()

def pretty_print_POST(req):
    """
    At this point it is completely built and ready
    to be fired; it is "prepared".

    However pay attention at the formatting used in 
    this function because it is programmed to be pretty 
    printed and may differ from the actual request.
    """
    print('{}\n{}\n{}\n\n{}'.format(
        '-----------START-----------',
        req.method + ' ' + req.url,
        '\n'.join('{}: {}'.format(k, v) for k, v in req.headers.items()),
        req.body,
    ))

pretty_print_POST(prepared)

-----------START-----------
POST https://gateway.watsonplatform.net/discovery/api/v1/environments/162e8398-ceff-42ab-8710-655087535953/collections/6cd34257-ac8c-43d5-b62f-61e0e9732eca/training_data?version=2018-03-05
content-type: application/json
Accept: text/plain
Content-Length: 156

natural_language_query=what+are+dos+and+donts+during+earthquake&filter=test2&examples=document_id&examples=relevance&examples=document_id&examples=relevance
